<a href="https://colab.research.google.com/github/gzmason/Relation-Extraction-Dataset-Finetuning/blob/Improved-Entity-Recognition/notebooks/entity_recognition_pos_tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pubmed_parser as pp
import spacy
path_xml = pp.list_xml_path('data') # list all xml paths under directory
pubmed_dict = pp.parse_medline_xml(path_xml[1]) 

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [ ]:
#nlp = spacy.load("en_core_sci_scibert")
nlp = spacy.load("en_core_sci_lg")

In [ ]:
!pip install "/content/scispacy_model/en_core_sci_lg-0.5.1.tar.gz"

Processing ./scispacy_model/en_core_sci_lg-0.5.1.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for en-core-sci-lg: filename=en_core_sci_lg-0.5.1-py3-none-any.whl size=532571818 sha256=45f69bb2610bc6f5bb6bc88a0007d038480d4b49fef79d2af20ca03e0299f44b
  Stored in directory: /root/.cache/pip/wheels/04/e3/e0/d889b3a6e381f8c224a1760e1eb8743627ff7444d71b96ece1
Successfully built en-core-sci-lg


#Mason#

In [ ]:
text_input = pubmed_dict[1]['abstract']
#text_input = 'Moreover, the presence of both hemin and a xenosiderophore resulted in (i) the strong induction of the expression of the proteins of the added xenosiderophore uptake pathway, (ii) repression of the PVD- and PCH-dependent iron uptake pathways, and (iii) no effect on the expression levels of the Has, Phu, or Hxu systems, indicating that bacteria use both xenosiderophores and heme to access iron.'
doc = nlp(text_input)

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/autocast_mode.py:117: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


In [ ]:
text_entity_list = []
for sentence in doc.sents:
  sentence_entity_list = []
  entity_start = -1
  entity_end = -1
  and_keyword_connecting = False

  for token in sentence:
    if (token.ent_type != 0 and token.tag_ == 'JJ'):
      if entity_start == -1: # Entering a new entity
        entity_start = token.idx
    elif (token.ent_type != 0 and token.pos_ == 'NOUN'):
      if entity_start == -1: # Entering a new entity
        entity_start = token.idx
      entity_end = token.idx + len(token) # Possible to end an entity here
    elif (token.text == "of"): #ignore the previous entity
      entity_start = -1
      entity_end = -1
    elif (token.text == "and" and entity_end == -1): #if not yet reaches the end of an entity
      continue
    else: 
      #checks if an entity has been formed
      if (entity_start != -1 and entity_end != -1):
        if and_keyword_connecting and (entity_end - sentence_entity_list[-1]['end'] <= 30): 
          #if close enough with previous entity and connected by "AND"
          sentence_entity_list[-1]['end'] = entity_end
        else: 
          #if is an independent entity
          sentence_entity_list.append({"start": entity_start, "end": entity_end, "label": "Entity"})
        and_keyword_connecting = False

      elif token.pos_ == "NOUN" and and_keyword_connecting:
        #if is a noun (but not recognized as an entity) and should be connected by AND keyword
        if (token.idx + len(token) - sentence_entity_list[-1]['end'] <= 30):
          sentence_entity_list[-1]['end'] = token.idx + len(token)
        and_keyword_connecting = False
      
      if token.text == 'and':
        and_keyword_connecting = True

      entity_start = -1
      entity_end = -1

  if (len(sentence_entity_list) > 0):
    text_entity_list.append(sentence_entity_list[0])
  if (len(sentence_entity_list) > 1):
    text_entity_list.append(sentence_entity_list[-1])


ex = {
    "text": doc.text,
    "ents": text_entity_list,
    "title": None
}

In [ ]:
spacy.displacy.render(ex, style='ent', manual=True, jupyter=True)

#Jason#

In [ ]:
pubmed_dict[1]['abstract']

'The current COVID-19 pandemic has claimed millions of lives all across the globe, making death more salient to many who may not have been readily cognizant of their mortality. While employees in certain occupations routinely deal with the idea of death or mortality (e.g., hospital workers, firefighters, and police officers), it is uncommon for the average employee to be within an environment that makes them aware of death. However, death awareness has been found to be negatively related to many important outcomes for the organization, including creativity. In the present study, using four-wave longitudinal data collected weekly-during late-June to late-July, 2020, we examine how employees react during the initial peak of COVID-19 pandemic in the United States in terms of death anxiety and death reflection (two different reactions to death awareness) and whether or not death anxiety and death reflection are related to creativity. Conducting cross-lagged panel modeling on four-wave long

In [ ]:
doc = nlp(pubmed_dict[1]['abstract'])

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/autocast_mode.py:117: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


Choose right word of "of" and concatenate words before and after "and". Only choose first word and last word from a sentence.

In [ ]:
ents_parsed = []
for sent in doc.sents:
  sentence_parsed = [{"text":token.text,"start":token.idx, "end": token.idx + len(token), "label": "entity", "pos": token.pos_ } for token in sent if (token.ent_type != 0 and (token.pos_ == "NOUN" or token.tag_ == "JJ")) or token.text in ["of", "and"]]
  tmp1 = []
  flag = False
  for ent in sentence_parsed:
    if ent["text"] in ["of","and"]:
      tmp1.append(ent)
      flag = True
    elif not tmp1 or flag or tmp1[-1]["end"] + 1 < ent["start"]:
      tmp1.append(ent) 
      flag = False
    else:
      tmp1[-1]["end"] = ent["end"]
      tmp1[-1]["text"] += " " + ent["text"]
      tmp1[-1]["pos"] = ent["pos"]

  flag = False
  tmp2 = []
  for ent in tmp1:
    if ent["text"] == "of":
      if tmp2 and tmp2[-1]["end"] + 1 == ent["start"]:
        tmp2.pop()
    elif ent["text"] == "and" and tmp2:
      if tmp2[-1]["end"] + 1 == ent["start"]:
        tmp2[-1]["end"] = ent["end"]
        tmp2[-1]["text"] += " " + ent["text"]
        flag = True 
    elif flag:
      tmp2[-1]["end"] = ent["end"]
      tmp2[-1]["text"] += " " + ent["text"]
      flag = False
    else:
      tmp2.append(ent)

  final_sentence_res = [tmp2[0], tmp2[-1]] if len(tmp2) > 1 else tmp2
  print(final_sentence_res)
  ents_parsed += final_sentence_res
customerized_ent = {
    "text": doc.text,
    "ents": ents_parsed,
    "title": None
}

[{'text': 'COVID-19 pandemic', 'start': 12, 'end': 29, 'label': 'entity', 'pos': 'NOUN'}, {'text': 'mortality', 'start': 165, 'end': 174, 'label': 'entity', 'pos': 'NOUN'}]
[{'text': 'employees', 'start': 182, 'end': 191, 'label': 'entity', 'pos': 'NOUN'}, {'text': 'death', 'start': 420, 'end': 425, 'label': 'entity', 'pos': 'NOUN'}]
[{'text': 'death awareness', 'start': 436, 'end': 451, 'label': 'entity', 'pos': 'NOUN'}, {'text': 'creativity', 'start': 551, 'end': 561, 'label': 'entity', 'pos': 'NOUN'}]
[{'text': 'study', 'start': 578, 'end': 583, 'label': 'entity', 'pos': 'NOUN'}, {'text': 'creativity', 'start': 931, 'end': 941, 'label': 'entity', 'pos': 'NOUN'}]
[{'text': 'cross-lagged panel modeling', 'start': 954, 'end': 981, 'label': 'entity', 'pos': 'NOUN'}, {'text': 'creativity', 'start': 1160, 'end': 1170, 'label': 'entity', 'pos': 'NOUN'}]
[{'text': 'theory and practice', 'start': 1215, 'end': 1234, 'label': 'entity', 'pos': 'NOUN'}]
[{'text': 'PsycInfo', 'start': 1237, 'end'

In [ ]:
text_entity_list = []
for sentence in doc.sents:
  sentence_entity_list = []
  entity_start = -1
  entity_end = -1
  and_keyword_connecting = False

  for token in sentence:
    if (token.ent_type != 0 and token.tag_ == 'JJ'):
      if entity_start == -1: # Entering a new entity
        entity_start = token.idx
    elif (token.ent_type != 0 and token.pos_ == 'NOUN'):
      if entity_start == -1: # Entering a new entity
        entity_start = token.idx
      entity_end = token.idx + len(token) # Possible to end an entity here
    elif (token.text == "of"): #ignore the previous entity
      entity_start = -1
      entity_end = -1
    elif (token.text == "and" and entity_end == -1): #if not yet reaches the end of an entity
      continue
    else: 
      #checks if an entity has been formed
      if (entity_start != -1 and entity_end != -1):
        if and_keyword_connecting and (entity_end - sentence_entity_list[-1]['end'] <= 30): 
          #if close enough with previous entity and connected by "AND"
          sentence_entity_list[-1]['end'] = entity_end
        else: 
          #if is an independent entity
          sentence_entity_list.append({"start": entity_start, "end": entity_end, "label": "Entity"})
        and_keyword_connecting = False

      elif token.pos_ == "NOUN" and and_keyword_connecting:
        #if is a noun (but not recognized as an entity) and should be connected by AND keyword
        if (token.idx + len(token) - sentence_entity_list[-1]['end'] <= 30):
          sentence_entity_list[-1]['end'] = token.idx + len(token)
        and_keyword_connecting = False
      
      if token.text == 'and':
        and_keyword_connecting = True

      entity_start = -1
      entity_end = -1

  if (len(sentence_entity_list) > 0):
    text_entity_list.append(sentence_entity_list[0])
  if (len(sentence_entity_list) > 1):
    text_entity_list.append(sentence_entity_list[-1])


ex = {
    "text": doc.text,
    "ents": text_entity_list,
    "title": None
}

In [ ]:
customerized_ent

{'ents': [{'end': 29,
   'label': 'entity',
   'pos': 'NOUN',
   'start': 12,
   'text': 'COVID-19 pandemic'},
  {'end': 174,
   'label': 'entity',
   'pos': 'NOUN',
   'start': 165,
   'text': 'mortality'},
  {'end': 191,
   'label': 'entity',
   'pos': 'NOUN',
   'start': 182,
   'text': 'employees'},
  {'end': 425,
   'label': 'entity',
   'pos': 'NOUN',
   'start': 420,
   'text': 'death'},
  {'end': 451,
   'label': 'entity',
   'pos': 'NOUN',
   'start': 436,
   'text': 'death awareness'},
  {'end': 561,
   'label': 'entity',
   'pos': 'NOUN',
   'start': 551,
   'text': 'creativity'},
  {'end': 583,
   'label': 'entity',
   'pos': 'NOUN',
   'start': 578,
   'text': 'study'},
  {'end': 941,
   'label': 'entity',
   'pos': 'NOUN',
   'start': 931,
   'text': 'creativity'},
  {'end': 981,
   'label': 'entity',
   'pos': 'NOUN',
   'start': 954,
   'text': 'cross-lagged panel modeling'},
  {'end': 1170,
   'label': 'entity',
   'pos': 'NOUN',
   'start': 1160,
   'text': 'creativit

In [ ]:
spacy.displacy.render(customerized_ent, style='ent', manual=True, jupyter=True)

Only choose noun and JJ and concatenate

In [ ]:
ents_parsed = [{"text":token.text,"start":token.idx, "end": token.idx + len(token), "label": "entity", "pos": token.pos_ } for token in doc if (token.ent_type != 0 and (token.pos_ == "NOUN" or token.tag_ == "JJ"))]
res = []
for ent in ents_parsed:
  if not res or res[-1]["end"] + 1 < ent["start"]:
    res.append(ent) 
  else:
    res[-1]["end"] = ent["end"]
customerized_ent = {
    "text": doc.text,
    "ents": res,
    "title": None
}

In [ ]:
spacy.displacy.render(customerized_ent, style='ent', manual=True, jupyter=True)

Original



In [ ]:
spacy.displacy.render(doc, style='ent', jupyter=True)

#Vanessa#


In [ ]:
text_input = pubmed_dict[1]['abstract']
#text_input = 'Moreover, the presence of both hemin and a xenosiderophore resulted in (i) the strong induction of the expression of the proteins of the added xenosiderophore uptake pathway, (ii) repression of the PVD- and PCH-dependent iron uptake pathways, and (iii) no effect on the expression levels of the Has, Phu, or Hxu systems, indicating that bacteria use both xenosiderophores and heme to access iron.'
doc = nlp(text_input)

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/autocast_mode.py:117: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


In [ ]:
text_entity_list = []
for sentence in doc.sents:
  sentence_entity_list = []
  entity_start = -1
  entity_end = -1
  and_keyword_connecting = False
  prev_adp = False

  for token in sentence:
    if (token.ent_type != 0 and token.tag_ == 'JJ'):
      if entity_start == -1: # Entering a new entity
        entity_start = token.idx
    elif (token.ent_type != 0 and token.pos_ == 'NOUN'):
      if entity_start == -1: # Entering a new entity
        entity_start = token.idx
      entity_end = token.idx + len(token) # Possible to end an entity here
    elif token.ent_type != 0 and not prev_adp:
      entity_start = -1
      entity_end = -1
      prev_adp = False
    elif (token.text == "of"): #ignore the previous entity
      entity_start = -1
      entity_end = -1
    elif (token.pos_ == 'ADP'):
      prev_adp = True
    elif (token.text == "and" and entity_end == -1): #if not yet reaches the end of an entity
      continue
    else: 
      #checks if an entity has been formed
      if (entity_start != -1 and entity_end != -1):
        if and_keyword_connecting and (entity_end - sentence_entity_list[-1]['end'] <= 30): 
          #if close enough with previous entity and connected by "AND"
          sentence_entity_list[-1]['end'] = entity_end
        else: 
          #if is an independent entity
          sentence_entity_list.append({"start": entity_start, "end": entity_end, "label": "Entity"})
        and_keyword_connecting = False

      elif token.pos_ == "NOUN" and and_keyword_connecting:
        #if is a noun (but not recognized as an entity) and should be connected by AND keyword
        if (token.idx + len(token) - sentence_entity_list[-1]['end'] <= 30):
          sentence_entity_list[-1]['end'] = token.idx + len(token)
        and_keyword_connecting = False
      
      if token.text == 'and':
        and_keyword_connecting = True

      entity_start = -1
      entity_end = -1


  for ent in sentence_entity_list:
    text_entity_list.append(ent)



ex = {
    "text": doc.text,
    "ents": text_entity_list,
    "title": None
}

In [ ]:
spacy.displacy.render(ex , style='ent', manual=True, jupyter=True)

In [ ]:
spacy.displacy.render(doc, style='ent', jupyter=True)

# Improved Version of Entity Recognition

In [ ]:
!pip install inflect

In [ ]:
import inflect
inflect_converter = inflect.engine()

'resource'

In [ ]:
import pickle
with open("/content/simple_dict.txt","rb") as file:
  simple_dict = pickle.load(file)

def entity_extraction(text_input):
  doc = nlp(text_input)
  text_entity_list = []
  for sentence in doc.sents:
    sentence_entity_list = []
    entity_start = -1
    entity_end = -1
    and_keyword_connecting = False
    prev_adp = False
    parenthesis_count = 0

    for token in sentence:
      if (token.ent_type != 0 and token.tag_ == 'JJ'):
        if entity_start == -1: # Entering a new entity
          entity_start = token.idx
      elif (token.ent_type != 0 and token.pos_ == 'NOUN'):
        if entity_start == -1: # Entering a new entity
          entity_start = token.idx
        entity_end = token.idx + len(token) # Possible to end an entity here
      elif token.ent_type != 0 and not prev_adp:
        entity_start = -1
        entity_end = -1
        prev_adp = False
      elif (token.text == "of"): #ignore the previous entity
        entity_start = -1
        entity_end = -1
      elif (token.pos_ == 'ADP'):
        prev_adp = True
      elif (token.text == "and" and entity_end == -1): #if not yet reaches the end of an entity
        continue
      else: 
        #checks if an entity has been formed
        if (entity_start != -1 and entity_end != -1):
          if and_keyword_connecting and (entity_end - sentence_entity_list[-1]['end'] <= 30): 
            #if close enough with previous entity and connected by "AND"
            sentence_entity_list[-1]['end'] = entity_end
          else: 
            #if is an independent entity and not inside a parenthesis
            if parenthesis_count == 0:
              sentence_entity_list.append({"start": entity_start, "end": entity_end, "label": "Entity"})
          and_keyword_connecting = False

        elif token.pos_ == "NOUN" and and_keyword_connecting:
          #if is a noun (but not recognized as an entity) and should be connected by AND keyword
          if (token.idx + len(token) - sentence_entity_list[-1]['end'] <= 30):
            sentence_entity_list[-1]['end'] = token.idx + len(token)
          and_keyword_connecting = False
        
        if token.text == 'and':
          and_keyword_connecting = True

        entity_start = -1
        entity_end = -1

      # If we enter a parenthesis, all independent entities inside it should not be considered
      if (token.text == "("):
        parenthesis_count = parenthesis_count + 1
      elif (token.text == ")"):
        parenthesis_count = parenthesis_count - 1

    for ent in sentence_entity_list:
      start_index = ent['start']
      end_index = ent['end']
      entity_text = doc.text[start_index:end_index].lower()

      """
      #Only consider it to be an entity if it is not a simple word
      if entity_text not in simple_dict:
        text_entity_list.append(ent)

      """
      #Only consider it to be an entity if at least one word is not a simple word
      should_add = False
      entity_word_list = entity_text.split(' ')
      for entity_word in entity_word_list:
        #Check both the original word and its singular form
        if entity_word not in simple_dict and inflect_converter.singular_noun(entity_word) not in simple_dict:
          should_add = True
      if should_add:
        text_entity_list.append(ent)
      
  ex = {
      "text": doc.text,
      "ents": text_entity_list,
      "title": None
  }

  spacy.displacy.render(ex , style='ent', manual=True, jupyter=True)

  return ex

In [ ]:
ex = entity_extraction(pubmed_dict[10]['abstract'])